In [ ]:
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import scipy as sc
import numpy as np
import requests as req
from selenium import webdriver
PATH = 'C:\Program Files (x86)\chromedriver.exe'
from selenium.webdriver.common.keys import Keys
import time
import matplotlib as mpl
import matplotlib.pyplot as plt      
import math
from collections import Counter
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, f1_score
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

steam_pages = 100
metacritic_pages = 125 #from 125

In [ ]:
count=0
h = "https://store.steampowered.com/search/?category1=998&os=win&filter=topsellers"
prices = list()

# selenium - scrolling down on steam:
driver=webdriver.Chrome(PATH)
driver.get(h)
html = driver.find_element_by_tag_name('html')
time.sleep(1)
previous_height = driver.execute_script('return document.body.scrollHeight;')
for x in range (1,steam_pages):
# while True:
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight;')
    if new_height == previous_height:
        break
    previous_height = new_height
html_final=html.get_attribute('innerHTML')

In [ ]:
def load_soup_object(html_doc):
    soup = bs(html_doc,"html.parser")
    return soup

# crawling:
soup = load_soup_object(html_final)
inner_soup = soup.find( id ='search_resultsRows' )

game_title = soup.findAll('span' , attrs={'class':'title'})
game_release_date = soup.findAll('div' , attrs={'class':'col search_released responsive_secondrow'})
game_price = soup.findAll( 'div' , attrs = {'class':'search_price'})
game_review = soup.findAll( 'div' , attrs = {'class':'col search_reviewscore responsive_secondrow'})
Win_platform = soup.findAll( 'span' , attrs = {'class':'platform_img win'})

#get inner games links:
inner_games_list = list() 
for link in inner_soup.find_all('a'):
    inner_games_list.append(link.get('href'))


In [ ]:
#get inner game html:
game_html = list()
game_developers = list()
game_details = list()
game_metascore = list()
game_storage = list()
labels = list()
clean_game_developers = list()
clean_game_details = list()
clean_game_metascore = list()
clean_game_storage = list()


i=0
for game in inner_games_list:
    game_html.append(req.get(game))
    game_soup = bs(game_html[i].text,"html.parser")
    game_developers.append(game_soup.find( id='developers_list' ))
    game_metascore.append(game_soup.find( id='game_area_metascore' ))
    game_storage.append(game_soup.findAll('div', attrs={'class':'game_area_sys_req'}))
    game_details.append(game_soup.find(id = 'genresAndManufacturer'))
    labels.append(game_soup.findAll('div' , attrs={'class':'label'}))
    i+=1
    

In [ ]:
#cleaning developers:
i=0
str_dev_list = [str(i) for i in game_developers]

for dev in game_developers:
    clean_dev = bs(str_dev_list[i] , "lxml").text
    clean_game_developers.append(clean_dev.strip())
    i+=1

#getting details with junk:
i=0
str_pub_list = [str(i) for i in game_details]

for pub in game_details:
    clean_details = bs(str_pub_list[i] , "lxml").text
    clean_game_details.append(clean_details.strip())
    i+=1

#getting metascore:
i=0
str_metascore = [str(i) for i in game_metascore]

for pub in game_metascore:
    clean_metascore = bs(str_metascore[i] , "lxml").text
    clean_game_metascore.append(clean_metascore.strip())
    i+=1    

#getting storage:
i=0
str_storage = [str(i) for i in game_storage]

for pub in game_storage:
    clean_storage = bs(str_storage[i] , "lxml").text
    clean_game_storage.append(clean_storage.strip())
    i+=1


In [ ]:
#find lables:

onlineCoop = list()
lanCoop=list()
singlePlayer=list()
onlinePVP=list()
fullControlerSupport=list()
patrialControllerSupport=list()
inAppPurch=list()
MMO=list()
SSCPVP=list()
j=0
for l in labels:
    str_storage = [str(i) for i in labels]
    r1 = str_storage[j].find('Online Co-op')
    r2 = str_storage[j].find('LAN Co-op')
    r3 = str_storage[j].find('Single-player')
    r4 = str_storage[j].find('Online PvP')
    r5 = str_storage[j].find('Full controller support')
    r6 = str_storage[j].find('Partial Controller Support')
    r7 = str_storage[j].find('In-App Purchases')
    r8 = str_storage[j].find('MMO')
    r9 = str_storage[j].find('Shared/Split Screen PvP')

    if r1 >= 0:
        onlineCoop.append(1)
    else:
        onlineCoop.append(0)
    if r2 >= 0:
        lanCoop.append(1)
    else:   
        lanCoop.append(0)
    if r3 >= 0:
        singlePlayer.append(1)
    else:   
        singlePlayer.append(0)
    if r4 >= 0:
        onlinePVP.append(1)
    else:   
        onlinePVP.append(0)
    if r5 >= 0:
        fullControlerSupport.append(1)
    else:   
        fullControlerSupport.append(0)
    if r6 >=0:
        patrialControllerSupport.append(1)
    else:
        patrialControllerSupport.append(0)
    if r7 >=0:
        inAppPurch.append(1)
    else:
        inAppPurch.append(0)
    if r8 >=0:
        MMO.append(1)
    else:
        MMO.append(0)
    if r9 >=0:
        SSCPVP.append(1)
    else:
        SSCPVP.append(0)
    
    j+=1


In [ ]:
#creating the df:
df=pd.DataFrame({'GameTitles':game_title , 'ReleaseYear':game_release_date , 'Price':game_price ,'Developer':clean_game_developers,
                 'Publisher':clean_game_details , 'Genre':clean_game_details,'Franchise': clean_game_details,
                 'Storage(GB)':clean_game_storage , 'SinglePlayer':singlePlayer,'onlineCo-op':onlineCoop,
                 'lanCo-op':lanCoop,'onlinePVP':onlinePVP,'FullControllerSupport':fullControlerSupport,'partialControllerSupport':patrialControllerSupport,
                 'inAppPurchases':inAppPurch,'MMO':MMO,'sharedSplitScreenPVP':SSCPVP,
                'SteamPositivePrecent':game_review , 'SteamUserReviews':game_review ,'Metascore':clean_game_metascore })
df.index+=1

# df.to_csv('rowDF.csv')
